# **C**hain of **A**gent with chain of **V**erific**A**tion (CAVA)


## Install dependencies

In [1]:
!pip install -qU langchain langchain-core langchain-text-splitters langchain-community langgraph langchain_chroma langchain-huggingface langsmith
!pip install -qU pypdf
!pip install -qU langchain-google-genai
# !pip install -qU optimum

In [2]:
#
from google.colab import userdata
import os
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# # Create LLM Model
# from langchain.chat_models import init_chat_model

# llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [4]:
# from huggingface_hub import login
# import os

# token = os.environ["HUGGINGFACE_HUB_TOKEN"]
# login(token=token)

In [5]:
# from huggingface_hub import notebook_login
# os.environ["HUGGINGFACE_HUB_TOKEN"] = userdata.get('HUGGINGFACE_HUB_TOKEN')
# notebook_login()

In [6]:
CHUNK_SIZE = 2000
MAX_NEW_TOKENS = 128 # [TODO] ??create two pipelines with diff max_new_tokens if VRAM allows
VERIFICATION_MODE = "every_k" # "none" | "every" | "every_k"
VERIFICATION_K = 3


In [7]:
# import sys
# print(sys.executable)

# import transformers, optimum
# print("transformers:", transformers.__version__)
# print("optimum:", optimum.__version__)


In [8]:
import torch
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# from awq import AutoAWQForCausalLM
from langchain_core.runnables import RunnableLambda


# Disable grad for torch
torch.set_grad_enabled(False)

def load_local_llm(model_id, max_new_tokens=MAX_NEW_TOKENS):
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # if "AWQ" in model_id:
    #     model = AutoAWQForCausalLM.from_pretrained(
    #         model_id,
    #         device_map="auto",
    #         # dtype=torch.float16,
    #         fuse_layers=True,
    #         trust_remote_code=True,
    #     )
    # else:
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype="auto",
        # dtype=torch.float16,
    )
    model.eval()
    def _generate(prompt: str) -> str:
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        with torch.inference_mode():
            out = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                use_cache=True,
            )
        # slice off prompt tokens
        gen_ids = out[0, inputs["input_ids"].shape[1]:]
        text = tokenizer.decode(gen_ids, skip_special_tokens=True)
        return text.strip()

    return RunnableLambda(lambda x: _generate(x))
    # pipe = pipeline(
    #     "text-generation",
    #     model=model,
    #     tokenizer=tokenizer,
    #     max_new_tokens=max_new_tokens,
    #     return_full_text=False,
    #     model_kwargs={"use_cache": True},
    # )

    # return HuggingFacePipeline(pipeline=pipe)

# llm_strong = load_local_llm("Qwen/Qwen2.5-32B-Instruct-AWQ", max_new_tokens=MAX_NEW_TOKENS)
llm_strong = load_local_llm("Qwen/Qwen2.5-14B-Instruct", max_new_tokens=MAX_NEW_TOKENS)
# llm_fast = load_local_llm("Qwen/Qwen2.5-7B-Instruct", max_new_tokens=MAX_NEW_TOKENS)

# worker = llm_fast
# manager = llm_strong
# verifier = llm_strong
# extractor = llm_fast

worker = llm_strong
manager = llm_strong
verifier = llm_strong
extractor = llm_strong

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [10]:
# print(llm_strong.pipeline.model.hf_device_map)
# print(llm_fast.pipeline.model.hf_device_map)

In [12]:
o = llm_strong.invoke("Testing testing testing")

In [13]:
# o = llm_fast.invoke("Testing testing testing")

In [14]:

# # model_id = "meta-llama/Llama-3.1-8B-Instruct"
# model_id = "Qwen/Qwen2.5-7B-Instruct"

# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     device_map="auto",
#     torch_dtype="auto",
# )

# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=MAX_NEW_TOKENS, # 32
#     return_full_text=False,
# )


# # llm = HuggingFacePipeline(pipeline=pipe)

# print(llm.invoke("What is an LLM?"))


In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=int(CHUNK_SIZE*0.1),
    separators=["\n\n", "\n", ". ", " "]
)
original_long_text = "testing split text" * 10

chunks = splitter.split_text(original_long_text)
chunks

['testing split texttesting split texttesting split texttesting split texttesting split texttesting split texttesting split texttesting split texttesting split texttesting split text']

In [16]:
# # Split pure text

# def split_text(text, chunk_size=500):
#     chunks = []
#     chunk_idx = 0
#     while chunk_idx < len(text):
#         end_idx = min(chunk_idx+chunk_size, len(text))
#         chunks.append(text[chunk_idx:end_idx])
#         chunk_idx = end_idx
#     return chunks
#     # return splitter.split_documents(text)

# original_long_text = "testing split text" * 10
# split_long_text = split_text(original_long_text, chunk_size=50)
# split_long_text

In [92]:
# --- Prompts ------------------------------------------------------------------
# For QA Tasks
WORKER_PROMPT = lambda i, query, chunk, prev: f"""
You are Worker {i} in a chain solving a long-context question answering task.

Use ONLY:
- the current source text (chunk)
- the previous worker summary

Task:
- Write a new summary that combines:
  (a) all information from the previous summary that is relevant to the query, and
  (b) any new relevant information in the current chunk.
- If the current chunk adds no new relevant information, simply repeat the previous summary unchanged.

Constraints:
- Maximum length: about 300 tokens.
- Output only the new summary, no commentary about your process.

Query:
{query}

Current source text (CHUNK {i}):
{chunk}

Previous worker summary:
{prev}

Now output the combined summary:
"""

# WORKER_PROMPT = lambda i, query, chunk, prev: f"""
# You are Worker {i} in a chain solving a long-context task.
# ONLY use the provided chunk and previous message.
# You need to read current source text and summary of previous source text (if any),
# and generate a summary to include them both and that best helps answer the query.
# Keep ≤ 300 tokens. If no new info, forward previous message unchanged.

# Query: {query}
# Current source text: CHUNK {i} (do NOT reference other chunks):\n{chunk}\n
# Previous source text :\n{prev}
# """

MANAGER_PROMPT = lambda query, final_worker_json: f"""
You are the Manager in a HotpotQA question answering system.

Task:
- Read the summary of evidence.
- Reason briefly about the answer.
- Then output the final answer as a short span, try to find the closest answer.

Output format (very important):
1. First, write a short reasoning paragraph if needed.
2. On the LAST line of your response, write exactly:

   Final answer: <answer>

Rules for <answer>:
- Use the shortest possible span (a name, location, date, number, or "yes"/"no").
- For yes/no questions, answer exactly "yes" or "no".
- Do NOT add any text after <answer> on that line.
- Do NOT write anything after the "Final answer: ..." line (no notes, no extra sentences).

Query:
{query}

Summary of evidence:
{final_worker_json}
"""

# MANAGER_PROMPT = lambda query, final_worker_json: f"""
# You are the Manager. Read the summary and decide on the best possible answer.

# Query:
# {query}

# Summary of evidence:
# {final_worker_json}

# First, reason briefly about the answer.
# Then end with a line starting with: Final answer: <answer>

# Make sure the final answer line contains the minimal answer span that should be evaluated.
# """
# lambda query, final_worker_json: f"""
# You are the Manager. Synthesize the final answer.
# Please keep the final answer as short as possible and do not respond with full sentences.
# Just reply with the final answer.
# The source is too long and has been summarized. You need to answer based on the summary.

# Query: {query}
# Final worker Summary: {final_worker_json}
# """

# ===== CoVe =====
PLAN_VERIFICATIONS_PROMPT = lambda query, chunk, baseline_summary: f"""
You are verifying a summary used in a long-context QA pipeline.

Original Query: {query}

Source chunk: {chunk}

Baseline summary: {baseline_summary}

Task:
Generate a small list of concrete verification questions (2–4) that help check:
- factual correctness
- coverage of key information relevant to the query
- absence of unsupported claims
Return the verification questions as a numbered list.
"""


EXEC_VERIFICATIONS_PROMPT = lambda query, chunk, qa_block: f"""
You are answering verification questions about a summary for a long-context QA pipeline.

Original Query: {query}

Source chunk: {chunk}

Here is a list of verification questions:
{qa_block}

For each question, answer concisely.
Formatting rules (very important):
- Return your answers as a **single numbered list**.
- Use exactly one line per answer.
- Do NOT repeat the list.
- Do NOT restate the questions.
- The format must be:

  1. <answer to Q1>
  2. <answer to Q2>
  3. <answer to Q3>
  ...
"""


GEN_FINAL_RESPONSE_PROMPT = lambda query, chunk, baseline_summary, questions, answers: f"""
You are revising a summary for a long-context QA pipeline.

Original Query: {query}

Source chunk: {chunk}

Baseline summary: {baseline_summary}

Verification Q&A:
{chr(10).join(f"Q: {q}\nA: {a}" for q, a in zip(questions, answers))}

Task:
Write a revised summary that:
- corrects any factual errors in the baseline summary
- adds missing key information supported by the source chunk
- removes unsupported or speculative claims
- remains concise and focused on information relevant to the question

Return ONLY the revised summary.
"""


EXTRACT_ANSWER_PROMPT = lambda query, manager_output: f"""
You are post-processing the output of a QA system on the HotpotQA dataset.

Your task: extract the **final answer string** that should be evaluated against the gold answer.

Constraints (very important):
- Return **only** the minimal answer span.
- Do **not** include explanations, reasoning, or extra words.
- Do **not** include phrases like "The answer is", "It is", "Final answer", etc.
- Do **not** add punctuation at the beginning or end unless it is part of the entity (e.g., "U.S.").
- Do **not** output multiple sentences.
- If the question is yes/no, answer with exactly **yes** or **no**.
- If the model’s answer is clearly wrong or missing, output exactly **no answer**.

Output format:
- Your entire response must be **only** the answer string, with no quotation marks and no additional text.

Query:
{query}

Model's answer:
{manager_output}

Now output the answer string only:
"""
# lambda query, manager_output: f"""
# You are post-processing the output of a QA system on the HotpotQA dataset.
# Your task: extract the **final answer string** that should be evaluated against the gold answer.
# Constraints (very important):
# - Return **only** the minimal answer span.
# - Do **not** include explanations, reasoning, or extra words.
# - Do **not** include phrases like "The answer is", "It is", etc.
# - Do **not** add punctuation at the beginning or end unless it is part of the entity (e.g., "U.S.").
# - Do **not** output multiple sentences.
# - If the question is yes/no, answer with exactly **"yes"** or **"no"**.
# - If the model’s answer is clearly wrong or missing, output **"no answer"**.

# HotpotQA style:
# - Answers are usually short spans: a name, location, date, number, or "yes"/"no".
# - Avoid extra context, titles, or clauses.

# Query: {query}
# Model's answer: {manager_output}

# Now output ONLY the final answer span that should be scored by HotpotQA:
# """

In [65]:
# Define agent graph
from typing import TypedDict, List

# [TODO]
class VerificationTrace(TypedDict):
    worker_idx: int
    baseline_summary: str
    verification_questions: List[str]
    verification_answers: List[str]
    verified_summary: str


class CoAState(TypedDict):
    query: str
    chunks: List[str]
    i: int
    worker_outputs: List[str]
    verbose: bool
    manager_output: str

    verification_mode: str # "none" | "every" | "every_k"
    verification_k: int
    store_verification_traces: bool
    verification_traces: List[VerificationTrace]


In [66]:
def worker_node(state: CoAState):
    i = state["i"]
    chunk = state["chunks"][i]
    if i == 0:
        prev = "No Previous summaries"
    else:
        # Get previous worker's output
        # print(state["worker_outputs"][i-1])
        # prev = state["worker_outputs"][i-1].content
        prev = state["worker_outputs"][i-1]
    prompt = WORKER_PROMPT(i, state["query"], chunk, prev)
    if state["verbose"]:
        print(f"Worker {i} with Prompt: \n######{prompt}\n#######\n")
        print("worker invoke")
    out = worker.invoke(prompt)

    if state["verbose"]:
        print("worker invoke -- done")
    # Note new outut
    state["worker_outputs"].append(out)
    state["i"] += 1
    if state["verbose"]:
        # print(f"Outputs: {out.content}\n------------------\n\n")
        print(f"Outputs: {out}\n------------------\n\n")

    return state

def manager_node(state:CoAState):
    if state["verbose"]:
        state["worker_outputs"][-1]
    # last_worker_output = state["worker_outputs"][-1].content
    last_worker_output = state["worker_outputs"][-1]
    prompt = MANAGER_PROMPT(state["query"], last_worker_output)
    if state["verbose"]:
        print(f"Manager with Prompt: \n######{prompt}\n#######\n")
    final_answer = manager.invoke(prompt)
    # store final summary as last output
    state["manager_output"] = final_answer
    if state["verbose"]:
        # print(f"Manager Final Output: \n#############\n{final_answer.content}")
        print(f"Manager Final Output: \n#############\n{final_answer}")

    return state


In [67]:
def parse_numbered_answers(exec_text: str, num_questions: int) -> List[str]:
    """
    Parse a numbered list like:
        1. Yes
        2) No
        3 - Maybe
    into ["Yes", "No", "Maybe"], capped at num_questions.

    Behavior:
    - Ignore ALL lines until the first line beginning with a number.
    - After that, parse consecutive numbered answers.
    - Stop after num_questions items.
    - Handle lines like "4. Yes 1. Yes..." by keeping only the first segment.
    """

    answers: List[str] = []
    started = False  # track when we hit first numbered line

    for line in exec_text.split("\n"):
        line = line.strip()
        if not line:
            continue

        # Detect first numbered line
        if not started:
            if line[0].isdigit():
                started = True
            else:
                continue  # skip until list starts

        # From here on, only accept numbered-list lines
        if not line[0].isdigit():
            continue

        # Strip the leading number (1., 1), 1 -, etc.)
        cleaned = re.sub(r"^\d+\s*[\.\)\-]\s*", "", line).strip()

        # Remove any second inline numbering (avoid "Yes 1. Yes, ..."), keep only the part before it
        parts = re.split(r"\s+\d+\s*[\.\)\-]\s*", cleaned)
        cleaned = parts[0].strip()

        if cleaned:
            answers.append(cleaned)
            if len(answers) >= num_questions:
                break

    # Fallback if no valid parsed answers
    if not answers:
        return [exec_text.strip()]

    return answers


def run_cove(query: str, chunk: str, baseline_summary: str, worker_idx: int, verbose: bool = False) -> VerificationTrace:
    # 1. Baseline response = baseline_summary (already produced by worker)
    # 2. Plan verification questions
    plan_prompt = PLAN_VERIFICATIONS_PROMPT(query, chunk, baseline_summary)

    if verbose:
        print(f"[CoVe][Worker {worker_idx}] Plan prompt:\n{plan_prompt}\n")

    plan_resp = verifier.invoke(plan_prompt)
    plan_text = str(getattr(plan_resp, "content", plan_resp)) # Depending on the LLM wrapper, llm.invoke() may return a plain string or a message object AIMessage(..., content="some text", ...)

    # crude parsing: split into lines that look like questions
    questions = [
        line.strip(" -0123456789.").strip()
        for line in plan_text.split("\n")
        if "?" in line
    ]
    questions = [q for q in questions if q]

    if verbose:
        print(f"[CoVe][Worker {worker_idx}] Verification Questions:\n{questions}\n")

    # 3. Execute verifications (factored: one call per question)
    answers = []
    qa_block = "\n".join(f"{i+1}. {q}" for i, q in enumerate(questions))
    exec_prompt = EXEC_VERIFICATIONS_PROMPT(query, chunk, qa_block)
    exec_resp = verifier.invoke(exec_prompt)
    exec_text = str(getattr(exec_resp, "content", exec_resp)).strip() #

    if verbose:
        print(f"verification q answer: {exec_text}")

    # answer parsing
    parsed_answers = parse_numbered_answers(exec_text, num_questions=len(questions))

    if len(parsed_answers) != len(questions):
        questions_for_gen = ["\n".join(questions)]
        answers_for_gen = [exec_text]
    else:
        questions_for_gen = questions
        answers_for_gen = parsed_answers

    if verbose:
        print(f"[CoVe][Worker {worker_idx}] Answers:\n{answers_for_gen}\n")

    # 4. Generate final verified summary
    final_prompt = GEN_FINAL_RESPONSE_PROMPT(query, chunk, baseline_summary, questions_for_gen, answers_for_gen)
    if verbose:
        print(f"final_prompt: {final_prompt}")
    final_resp = verifier.invoke(final_prompt)
    final_summary = str(getattr(final_resp, "content", final_resp)).strip() # [TODO] add in worker node? Depending on the LLM wrapper, llm.invoke() may return a plain string or a message object AIMessage(..., content="some text", ...)

    if verbose:
        print(f"[CoVe][Worker {worker_idx}] Final verified summary:\n{final_summary}\n")

    trace: VerificationTrace = {
        "worker_idx": worker_idx,
        "baseline_summary": baseline_summary,
        "verification_questions": questions,
        "verification_answers": answers,
        "verified_summary": final_summary,
    }
    return trace


def verification_node(state: CoAState, worker_idx: int):
    query = state["query"]
    chunk = state["chunks"][worker_idx]

    raw_summary = state["worker_outputs"][worker_idx]
    baseline_summary = str(getattr(raw_summary, "content", raw_summary)) #

    trace = run_cove(
        query=query,
        chunk=chunk,
        baseline_summary=baseline_summary,
        worker_idx=worker_idx,
        verbose=state["verbose"],
    )

    # replace worker summary with verified one
    state["worker_outputs"][worker_idx] = trace["verified_summary"]

    # store trace if needed
    if state.get("store_verification_traces", False): # defaults to False if the key doesn’t exist
        state["verification_traces"].append(trace)

    return state


def maybe_run_verification(state: CoAState) -> CoAState:
    """
    determine if the latest generated summary needs to be verified

    Param:
    state (returned by worker_node())

    Return:
    updated state
    """
    mode = state["verification_mode"] # "none" | "every" | "every_k"
    k = state["verification_k"]
    current_worker_idx = state["i"] - 1 # cuz in worker_node() before returning state it does state["i"] += 1

    if mode == "none":
        return state
    if mode == "every":
        return verification_node(state, current_worker_idx)
    if mode == "every_k" and (current_worker_idx + 1) % k == 0:
        return verification_node(state, current_worker_idx)

    return state

In [68]:
def run_coa(query, context, verbose=True, verification_mode="none", verification_k=1, store_verification_traces=True, postprocess=True): # chunk_size
    # Split context
    # chunks = split_text(context, chunk_size=chunk_size)
    chunks = splitter.split_text(context)
    if verbose:
        print("Text Chunks: ",chunks)
    # assert 1==2
    # Initialize initial CoAState
    init_state = {
        "query": query,
        "chunks": chunks,
        "i": 0,
        "worker_outputs": [],
        "verbose": verbose,
        "manager_output": "",
        # [CoVe]
        "verification_mode": verification_mode,
        "verification_k": verification_k,
        "store_verification_traces": store_verification_traces,
        "verification_traces": []
    }
    state = init_state
    length = len(chunks)
    if verbose:
        print("Num Chunks: ", length)
    # Worker nodes, for each chunk
    for i, chunk in enumerate(chunks):
        # Run worker node and get new state
        if verbose:
            print(f"Running Worker {i}")
        state = worker_node(state)
        if verbose:
            print(f"Running Worker {i} -- Done")
        # [TODO]
        if verbose:
            print(f"Verifying Worker {i}")
        state = maybe_run_verification(state)
        if verbose:
            print(f"Verifying Worker {i} -- Done")

    # At the end of the loop, state["i"] should be == len(chunks)
    assert state["i"] == len(chunks), "Total states worked does not equal to number of text chunks"

    # Finally run manager at last
    if verbose:
        print(f"Manager producing output")
    state = manager_node(state)
    # final_ans = state["worker_outputs"][-1].content
    final_ans = state["manager_output"]
    if verbose:
        print("Final Answer before process: ", final_ans)
    if "Final Answer: ".lower() in final_ans.lower():
        if verbose:
            print("splitting parsing")
        final_ans = final_ans.lower().split("Final answer: ".lower())[-1]
    if verbose:
        print(f"Manager producing output -- Done")

    # Post processing
    if postprocess and False:
        if verbose:
            print(f"Extractor")
        prompt = EXTRACT_ANSWER_PROMPT(query, state["manager_output"])
        resp = extractor.invoke(prompt)
        final_ans = str(getattr(resp, "content", resp)).strip()
        if verbose:
            print(f"Extractor -- Done")

    if verbose:
        print(f"Query: {state["query"]}\nFinal Answer: {final_ans}")
    return final_ans

In [22]:
# # Test run CoA
# ans = run_coa("what is the meaning?", original_long_text, chunk_size=50, verbose=False)
# ans

In [23]:
# ans

## Eval

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
# [TODO]
LOG_PATH = "/content/drive/MyDrive/GenAI/project/logs/hotpotqa_valtest.jsonl"
SUBSET_START = 0
SUBSET_END = 3
NUM_SAMPLES_TO_LOAD = SUBSET_END - SUBSET_START # may need to load more
SEED = 130

In [94]:
import json
from typing import List, Dict, Callable
from datasets import load_dataset
from tqdm import tqdm
import re
import string
import random

# -----------------------------------------------------------
# 1. HotpotQA Loader
# -----------------------------------------------------------

def load_hotpotqa(split="validation", max_samples=None):
    """
    [source] https://huggingface.co/datasets/hotpotqa/hotpot_qa

    an example in hotpotqa - fullwiki:
    {
        "id": str,
        "question": str,
        "answer": str,
        "type": str,
        "level": str,
        "supporting_facts":
        {
            "title": [str, str, ...], # may repeat
            "sent_id": [int32, int32, ...]
        },
        "context":
        {
            "title": [str, str, ...],
            "sentences": [[str, str, str, ...], [str, str, str, ...], ...]
        }

    }

    Return:
    a list of dicts
    {
        "id": HotpotQA string id,
        "idx": int index within this split (for convenience),
        "question": str,
        "answer": str,
        "type": "bridge" or "comparison",
        "level": "easy"/"medium"/"hard",
        "context":
        [
            { "title": str, "sentences": [str, str, ...] }, # doc 0
            { "title": str, "sentences": [str, str, ...] }, # doc 1
            ...
        ]
    }
    """
    raw = load_dataset("hotpot_qa", "fullwiki")[split]

    # Generate random indicies

    rng = random.Random(SEED)
    indices = list(range(len(raw)))
    rng.shuffle(indices)
    indicies = indices[:max_samples]
    print("Random sampled indicies: ", indicies)

    data = []
    for idx, item in enumerate(raw):
        context = [
            {
                "title": t,
                "sentences": sents
            }
            for t, sents in zip(item["context"]["title"], item["context"]["sentences"])
        ]

        data.append({
            "id": item["id"],                 # original HotpotQA id (string)
            "idx": idx,                       # integer position in this split
            "question": item["question"],
            "answer": item["answer"],
            "type": item.get("type"),         # [TODO] ?? "bridge" or "comparison"
            "level": item.get("level"),       # [TODO] ?? "easy"/"medium"/"hard"
            "context": context,
        })

        if max_samples and len(data) >= max_samples:
            break

    return data


# -----------------------------------------------------------
# 2. Context Merger
# -----------------------------------------------------------

def merge_context_fullwiki(context):
    """
    merge each document's sentence list into a single text string

    Param:
    context (return from load_hotpotqa()):
    [
        { "title": str, "sentences": [str, str, ...] }, # doc 0
        { "title": str, "sentences": [str, str, ...] }, # doc 1
        ...
    ]

    Return:
    texts: list[str]
    merged text for each document
    """
    texts = []

    for doc in context:
        text = " ".join(doc["sentences"])
        texts.append(text)

    return texts


# -----------------------------------------------------------
# 3. Evaluation Metrics (EM + F1)
# -----------------------------------------------------------

def normalize_answer(s):
    """
    Lowercase, remove punctuation/articles/extra whitespace.
    """
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


# TODO: does the paper compute this at the token level??
def f1_score(pred, gold):
    pred_tokens = normalize_answer(pred).split()
    gold_tokens = normalize_answer(gold).split()

    if len(pred_tokens) == 0 or len(gold_tokens) == 0:
        return int(pred_tokens == gold_tokens)

    common = set(pred_tokens) & set(gold_tokens)
    num_same = sum(min(pred_tokens.count(t), gold_tokens.count(t)) for t in common)

    if num_same == 0:
        return 0

    precision = num_same / len(pred_tokens)
    recall = num_same / len(gold_tokens)
    return (2 * precision * recall) / (precision + recall)


def exact_match(pred, gold):
    return normalize_answer(pred) == normalize_answer(gold)


# -----------------------------------------------------------
# 4. Evaluation loop
# -----------------------------------------------------------

def evaluate(model_fn: Callable, dataset: List[Dict], log_path: str | None = None, flush_every: int = 10, include_context_in_log: bool = False):
    """
    model_fn(query, context_chunks) -> str

    If log_path is provided, per-sample metrics will be written to a JSONL file, one JSON object per line.
    This allows:
      - parallel eval on disjoint subsets
      - later merging / error analysis
    """
    # qs = []
    # ctxs = []
    # preds = []
    # refs = []

    f1s = []
    ems = []
    N = 0

    # Open log file once (append mode so you can resume)
    log_file = None
    if log_path is not None:
        os.makedirs(os.path.dirname(log_path), exist_ok=True)
        log_file = open(log_path, "a", encoding="utf-8")

    try:
        for sample in tqdm(dataset, desc="Evaluating"):
            qid = sample.get("id")
            question = sample["question"]
            context = sample["context"]
            gold = sample["answer"]

            texts = merge_context_fullwiki(context)
            pred = model_fn(question, texts)

            f1 = f1_score(pred, gold)
            em = int(exact_match(pred, gold))

            f1s.append(f1)
            ems.append(em)
            N += 1

            # Per-sample record for logging
            record = {
                "id": qid,
                "idx": sample.get("idx"),
                "type": sample.get("type"),
                "level": sample.get("level"),
                "question": question,
                "gold_answer": gold,
                "prediction": pred,
                "f1": f1,
                "em": em,
            }
            if include_context_in_log:
                record["context"] = texts  # can be big; toggle with flag

            if log_file is not None:
                log_file.write(json.dumps(record, ensure_ascii=False) + "\n")

                # Periodic flush so progress is safely on disk
                if N % flush_every == 0:
                    log_file.flush()
                    os.fsync(log_file.fileno())

        # Final flush
        if log_file is not None:
            log_file.flush()
            os.fsync(log_file.fileno())

    finally:
        if log_file is not None:
            log_file.close()

    return {
        "num_samples": N,
        "f1": sum(f1s) / N if N > 0 else 0.0,
        "em": sum(ems) / N if N > 0 else 0.0,
    }


# -----------------------------------------------------------
# 5. Placeholder CoA model
# -----------------------------------------------------------

def coa_placeholder(question: str, context_texts: List[str]) -> str:
    """
    Dummy version to make the pipeline runnable now.
    Replace with Ray's CoA later.
    """
    merged_context = " ".join(context_texts)

    # print(merged_context)
    # prompt = f"Context:\n{merged_context}\n\nQuestion: {question}\nAnswer:"
    # TODO
    # print(f"Length of merged context: {len(merged_context)}")
    # return "hello"
    # assert 1==2
    # TODO: Figure out what chunk size is best cost to performance
    final_ans = run_coa(query=question, context=merged_context, verbose=True, verification_mode=VERIFICATION_MODE, verification_k=VERIFICATION_K, store_verification_traces=True)
    return final_ans
    return "PLACEHOLDER YES" # "PLACEHOLDER_ANSWER"


# -----------------------------------------------------------
# 6. Running the pipeline
# -----------------------------------------------------------

if __name__ == "__main__":
    # Full Hotpot QA size is 7405, takes ~ 29 hours with 512 chunk size
    data = load_hotpotqa(split="validation", max_samples=NUM_SAMPLES_TO_LOAD)
    subset = data[SUBSET_START:SUBSET_END]
    results = evaluate(coa_placeholder, subset, log_path=LOG_PATH, flush_every=5, include_context_in_log=False)



Random sampled indicies:  [2554, 1722, 6283]


Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

Text Chunks:  ['Adam Collis is an American filmmaker and actor.  He attended the Duke University from 1986 to 1990 and the University of California, Los Angeles from 2007 to 2010.  He also studied cinema at the University of Southern California from 1991 to 1997.  Collis first work was the assistant director for the Scott Derrickson\'s short "Love in the Ruins" (1995).  In 1998, he played "Crankshaft" in Eric Koyanagi\'s "Hundred Percent". Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood.  The film concerns the period in Wood\'s life when he made his best-known films as well as his relationship with actor Bela Lugosi, played by Martin Landau.  Sarah Jessica Parker, Patricia Arquette, Jeffrey Jones, Lisa Marie, and Bill Murray are among the supporting cast. Tyler Bates (born June 5, 1965) is an American musician, music producer, and composer for films, television, and video games.  Mu

Evaluating:  33%|███▎      | 1/3 [00:46<01:32, 46.28s/it]

Manager Final Output: 
#############
Final answer: yes

Note: The final answer should be based on the information provided in the summary. Since both individuals are associated with American film productions and there is no conflicting information regarding their nationalities, it can be reasonably inferred that they are of the same nationality, which is American. However, the exact confirmation for Scott Derrickson's nationality is not explicitly stated in the given text, but the inference is made based on the context provided. Given the instruction to provide the closest answer from the text, and considering the strong implication, "yes" is chosen as the final answer. Please note that this conclusion is based on the provided information
Final Answer before process:  Final answer: yes

Note: The final answer should be based on the information provided in the summary. Since both individuals are associated with American film productions and there is no conflicting information regarding 

Evaluating:  67%|██████▋   | 2/3 [01:28<00:44, 44.09s/it]

Manager Final Output: 
#############
not mention any government positions held by either actress.

Reasoning:
Based on the provided summary, there is no information available regarding any government position held by the actress who portrayed Corlss Archer in the film "Kiss and Tell."

Final answer: no
Final Answer before process:  not mention any government positions held by either actress.

Reasoning:
Based on the provided summary, there is no information available regarding any government position held by the actress who portrayed Corlss Archer in the film "Kiss and Tell."

Final answer: no
splitting parsing
Manager producing output -- Done
Query: What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?
Final Answer: no
Text Chunks:  ['Animorphs is a science fantasy series of young adult books written by Katherine Applegate and her husband Michael Grant, writing together under the name K. A. Applegate, and published by Scholastic.  It is

Evaluating: 100%|██████████| 3/3 [02:17<00:00, 45.76s/it]

Manager Final Output: 
#############
question asked.

Reasoning:
The provided evidence mentions Animorphs as a science fantasy young adult series that fits the first-person narration criteria. However, it does not mention any companion books that narrate the stories of enslaved worlds and alien species. Therefore, based on the given information, we cannot confirm if Animorphs is the correct answer or if there is another series that matches all the criteria.

Final answer: no

(Note: The response indicates that the given information does not confirm the existence of such a series, leading to a "no" answer based on the available evidence.) 

Final answer: no
Final answer: no
Final Answer before process:  question asked.

Reasoning:
The provided evidence mentions Animorphs as a science fantasy young adult series that fits the first-person narration criteria. However, it does not mention any companion books that narrate the stories of enslaved worlds and alien species. Therefore, based on 

In [91]:
results

{'num_samples': 3, 'f1': 0.0, 'em': 0.0}

In [81]:
results['f1']

0.3333333333333333

In [76]:
for i in range(3):
    print(":)")

:)
:)
:)


In [82]:

import json
print(LOG_PATH)
data = []
with open(LOG_PATH, "r") as f:
    for line in f:
        data.append(json.loads(line))
        print(json.loads(line))

print(len(data))
print(data[0])


/content/drive/MyDrive/GenAI/project/logs/hotpotqa_val003.jsonl
{'id': '5a8b57f25542995d1e6f1371', 'idx': 0, 'type': 'comparison', 'level': 'hard', 'question': 'Were Scott Derrickson and Ed Wood of the same nationality?', 'gold_answer': 'yes', 'prediction': 'yes', 'f1': 1.0, 'em': 1}
{'id': '5a8c7595554299585d9e36b6', 'idx': 1, 'type': 'bridge', 'level': 'hard', 'question': 'What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?', 'gold_answer': 'Chief of Protocol', 'prediction': 'no', 'f1': 0, 'em': 0}
{'id': '5a85ea095542994775f606a8', 'idx': 2, 'type': 'bridge', 'level': 'hard', 'question': 'What science fantasy young adult series, told in first person, has a set of companion books narrating the stories of enslaved worlds and alien species?', 'gold_answer': 'Animorphs', 'prediction': 'no\n\n(note: the response was generated based on the provided evidence, which did not contain the exact information required to answer the', 'f1': 0, 'em

In [ ]:
for i in range(3):
    print(f"[{i}] Questions:{results['questions'][i]}\nPrediction: {results['predictions'][i]}  ---> Reference: {results['references'][i]}\n\n")

In [ ]:
results['contexts'][0]

In [ ]:
results['questions'][0]

In [ ]:
results['references'][0]

In [ ]:
results['predictions'][0]

In [ ]:
# Testing committing from google colab